# SCI1022 Python workshop 3: 1D Motion under uniform gravitational field

Welcome to the third workshop of the Python stream! 

Some miscellaneous instructions to begin with:

* Use your two hours in this workshop to work through this Jupyter notebook. 

* Read the instructions as you go. Execute the example code cells provided, and look carefully at the output. 

* Recall that, in order to execute a code cell, then you can left-click on it (either on the interior or the left-hand-side margin of the cell), and then hit `Shift-Enter`, which stands for: "While holding pressed the `Shift` keyboard key, press the keyboard key labelled as `Enter`". 

* If you struggle trying to use the Jupyter notebook, then explore usage instructions and links available at the SCI1022 Python's Moodle page. 

* <span style="color:red">**Tasks**</span> are marked in <span style="color:red">**red**</span> and displayed in indented blocks. The tasks are solved by writing code on the blank code cells provided.

* Ask your instructors for help at any time.

---

Following biology and pure mathematics, the next few weeks we will focus on physics in order to improve your current code skills and learn new ones. In particular, we will do kinematics and simulate the **motion of objects**. 

*This week* we are going to work under the assumption that the objects in motion are going to be sufficiently close to the surface of Earth such that the gravitational force among the object and the Earth pulls down the objects with a uniform acceleration.

You will write code able to simulate the motion of a projectile thrown not too fast from the ground level. You will do it in guided baby steps, so don't panic if your physics are rusty! 

For simplicity, in this week we will start with motion in one dimension (1D), i.e., the object can only go move up or down. The next week we will extend the code to two-dimensional (2D) motion.

## NumPy arrays

In this tutorial we are going to introduce you to a new compound type; the "array". 

You can think of an array as a list, i.e., an ordered collection of elements.  As with lists, each element of an array is assigned a number - its position or index. The first index is zero, the second index is one, and so forth. 

However, there are three main differences among arrays and lists:

1. **An array can keep only elements of the same type**, preferably `int`, `float`, or `complex` numbers,
whereas a list can mix different kinds of objects. This means arrays are more efficient for efficient numerical computing and storage (because, among others, we don’t need to store the type for every element). It also makes arrays the data structure of choice for numerical calculations where we often deal with vectors.

2. **The number  of elements in an array must be known when the array is created.** Once created, the number of elements in an array *can* be changed, but at a substantial significant computational cost.

3. **arrays can be extended to arbitrary dimensions**. For example, a 2D array can be used to store a matrix, and thus it is indexed by a row index and a column index. However, by now, for simplicity, we will restrict ourselves to 1D arrays, and move to 2D arrays in further tutorials when we have gained enough familiarity with 1D ones.


When using the computer to perform numerical computations, we often end up with a vast amount of numbers and associated arithmetic operations. 

Using lists to store such data can lead to slow programs, while
arrays can make the programs run much faster. 

This is crucial for many advanced applications in scientific computing, where computer programs may run for hours and days, or even weeks.

Arrays are not built-in in Python. Fortunately, they are provided by a package referred to as *Numerical Python*, often abbreviated as [NumPy](https://numpy.org/).


### Quick overview of 1D NumPy arrays (vectors)

Let us briefly overview how working with 1D arrays looks like. For obvious reasons, 1D arrays are also known as **vectors**.
(In the sequel, execute the example code cells provided, and look carefully at the output.) The first thing you typically do with an array is to generate it. Here are a few examples of how we can generate one:

* Conversion of a list into a vector using the `array` function:

In [ ]:
import numpy
x = numpy.array([0, 0.5, 1, 1.5])
print(x)

* Create a vector of given size and initialize it with zeros using the `zeros` function:

In [ ]:
x = numpy.zeros(4)
print(x)

* Create a vector with evenly spaced values within a given interval using the `arange` function:


In [ ]:
x = numpy.arange(0, 1.1, 0.1)
print(x)

Note that, in this example, the space among values is $0.1$, and the interval is $[0.0,1.1)$ (left-closed, right-open).


Once the vector is established, we can set and retrieve individual values, and we can extract slices of the arrays using ranges, as with lists. For example:

In [ ]:
x = numpy.zeros(4)
x[0] = 3.4
x[2] = 4
print(x)
print(x[0])
print(x[0:-1])

### The concept of vectorization and how to exploit it with Numpy

You might recall, from high school math, vector operations such as: (1) summation or substraction of two vectors; (2) multiplication of a scalar by a vector; (3) scalar/dot product among vectors; (4) computing the length of the vector, to name a few. In addition to these operations, it turns out that in numerical computing it is of paramount importance to be able to support efficiently the application of a function (e.g., a mathematical expression) to all elements in a vector at once.

This is referred to as **vectorization**, **vector arithmetics** or **vector notation**. Numpy supports vector arithmetics for arrays of any dimension and shape. However, for simplicity, let us illustrate this concept with vectors (i.e., 1D arrays).

As seen in the code cell below, once we have generated a vector with Numpy, we can perform calculations on every element in the vector with a single statement (even for user-defined functions as double):

In [ ]:
import numpy

def double(x):
    return x*2

x = numpy.arange(0, 2, 0.5)
print(x)
print(x + 10)
print(x ** 2)
print(numpy.sin(x))
print(double(x))

We note that, in the previous cell, we are applying mathematical functions of one variable, e.g., `x+10`, `x**2`, or `sin(x)`, to a vector. The result is  a vector where the function is applied to each element of the vector separately.

The point here is that using vectorization we enable Numpy to be able to efficiently implement the loop that applies the user-defined function (e.g., `double`), a pre-defined function (e.g., `numpy.sin`), or mathematical expression (e.g., `x+2` or `x**2`) to the each of the entries of the vector.

**This is very useful for speeding up programs.**  Of course, we could implement the same operation using a hand-written `for` loop, and entry-wise access to the entries of the vector, but this would result in a significantly slower code.

NumPy is a vast package with many more features. However, what we have seen above is sufficient for the current tutorial. If you want to learn more (out of class), you can, e.g., go to https://numpy.org/doc/stable/user/absolute_beginners.html.

## 1D Motion: projectile thrown up from the ground level

In the rest of the tutorial, we are going to write a code that simulates the motion of a projectile thrown up at a given initial velocity from the ground level. 

We will restrict ourselves to 1D motion, i.e., the projectile only moves north (up) or south (down) in the vertical direction. 

We are interested into determine both the **displacement** of the projectile *with respect to the ground* (i.e., its height), and the **velocity** of the projectile, at each point in time. Recall that both **displacement** and **velocity** are vectors, and thus have a magnitude and a direction. In 1D, the sign indicates the direction of vector quantities. For the displacement we will adopt the convention that a positive sign means that the projectile is above the ground level, and positive and negative velocity, that it is moving north and south, respectively. Thus, for example, zero displacement means that the object is on the ground level, and a velocity of -5 $\frac{m}{s}$ that the projectile is moving down, where $m$ denotes "meters", and $s$ "seconds".

As mentioned at the beginning of the tutorial, we are going to work under the assumption that the projectile is going to be sufficiently close to the surface of Earth such that the gravitational force among the object and the Earth pulls down the objects with a uniform (i.e., constant) acceleration. Accordingly to Newton's universal law of gravitation, such constant acceleration is of 9.8 $\frac{m}{s^2}$ of magnitude. Recall that the acceleration is also a vector. The direction of the gravitational acceleration is always south (under the assumption above), and thus the acceleration caused by the gravitational force has a negative sign. In other words, the acceleration vector always points downwards.

As you might have guessed (from experience among others), the motion subject of study can be split into two stages:

1. **The projectile first moves north**. During this stage, the initial velocity at which the projectile is thrown, progressively decreases due to the acceleration (actually deceleration) of gravity, until at a point in time that the velocity reaches $0$ $\frac{m}{s}$, i.e., the projectile is instantaneously not moving. 

2. **The projectile then moves south**. During this stage, the velocity progressively increases (pointing towards the ground) due to the acceleration of gravity, until at a point in time that the projectile hits the ground. 

We denote as $\vec{x}(t)$ and $\vec{v}(t)$ the displacement and velocity of the projectile, respectively, $t$ seconds after throwing it upward. $T$ denotes the simulation time, i.e., we assume that we are going to simulate the process in the time interval $t \in [0,T]$. $\vec{v}_0=\vec{v}(0)$ is the initial velocity at which the projectile is thrown. If we apply basic kinematics, we have that the displacement and velocity are given by:

1. $\vec{x}(t) = \vec{v}_0 \cdot t - 9.8 \cdot \frac{t^2}{2}$
2. $\vec{v}(t) = \vec{v}_0 - 9.8 \cdot t$

Well, we are lucky to have Newton providing us with analytical expressions for the displacement and velocity **exactly** as a function of time. However, this is unfortunately not the case in the vast majority of real-world simulations, and we have instead to work with numerical approximations to simulate the problem at hand. This is the main point of this tutorial.

In order to build an approximation of $\vec{x}(t)$ and $\vec{v}(t)$ we are going to split the time interval $[0,T]$ into equally-sized time subintervals of size denoted as $\Delta t$, and **assume that the velocity is constant within each of these subintervals**. We will refer to  $\Delta t$ as the "time-step" size. Why is this just an approximation? As you can see in the equation above for $\vec{v}(t)$, **the velocity continuously changes with $t$**, but we are assuming it to be constant subinterval-wise. However, we will check using code that if $\Delta t$ is small enough, these approximations are good enough.

### Time splitting

Let us denote as $N$ the number of equally-sized time subintervals in which we are going to split $[0,T]$. 

><span style="color:red">**Task 1.**</span> 
Describe in words how you would compute $\Delta t$ as a function of $T$ and $N$. For example, if $T=20$ seconds, and $N=4$, then $\Delta t=5$ seconds.

<span style="color:red">**Write your answer to Task 1 in this cell.**</span>

The first time subinterval is $[0,\Delta t]$, the second is $[\Delta t,2\cdot\Delta t]$, the third subinterval is $[2\cdot\Delta t,3\cdot\Delta t]$, and so on. 

><span style="color:red">**Task 2.**</span> 
Assuming a simulation time of $T=20$ seconds, and $N=5$ subintervals, write down the five subintervals in which we split $[0,20]$.

<span style="color:red">**Write your answer to Task 2 in this cell.**</span>

Going more formally, let us assign a consecutive integer identifier to each subinterval starting from one. That is, we well assign the identifier $1$ to $[0,\Delta t]$,  the identifier $2$ to $[\Delta t,2\cdot\Delta t]$, etc. We will denote as $T_i$ the interval with identifier $i$, with $1 \leq i\leq N$. Thus, for example, $T_1=[0,\Delta t]$, $T_2=[\Delta t,2\cdot\Delta t]$, etc. 

><span style="color:red">**Task 3.**</span> 
What is the identifier of the last subinterval if we have $N$ subintervals? How we denote the last subinterval?

<span style="color:red">**Write your answers to Task 3 in this cell.**</span>

For a given time subinterval, $T_i$, we will denote as $t_{i-1}$ the lower bound of the interval, and $t_{i}$ as the upper bound. For example, for $T_1=[t_0,t_1]$, we have that $t_0=0$, and $t_1=\Delta t$. 

><span style="color:red">**Task 4.**</span> 
> * What are the lower and upper bounds for $T_3$ (i.e., the values of $t_2$ and $t_3$)?
> * Can you think of a general expression that given an identifier $i$ , provides the values of $t_{i-1}$ and $t_{i}$ as a function of $i$ and $\Delta t$? Check that the general expression works with the table below.

<span style="color:red">**Write your answers to Task 4 in this cell.**</span>

To end this section, and help you out, the table below summarizes most of the notation introduced so far for $N=4$, $T=1$, and thus $\Delta t = 0.25$.

| $i$  | $t_{i-1}$  | $t_i$   | $T_i$   |
|---|---|---|---|
| 1  | $0$  | $0.25$  | $[0,0.25]$  |
| 2  | $0.25$  | $0.5$  |$[0.25,0.5]$   |
| 3  | $0.5$  | $0.75$  | $[0.5,0.75]$  |
| 4  | $0.75$  | $1.0$ |$[0.75,1.0]$   |

### Moving time forward

We will denote as $\vec{\tilde{x_i}}$ and $\vec{\tilde{v_i}}$ the computed approximation of the displacement and velocity of the projectile at time $t_i$. (Note the tilde on top of $x_i$ and $v_i$). In other words, $\vec{\tilde{x_i}} \approx \vec{x_i}=\vec{x}(t_i)$, and $\vec{\tilde{v_i}} \approx \vec{v_i}=\vec{v}(t_i)$, where "$\approx$" denotes that two quantities are just an approximation of each other.

We will start the simulation with an initial displacement of $\vec{\tilde{x_0}}=0$ (the projectile is thrown from the ground level!) and given initial velocity $\vec{\tilde{v_0}}$. Then, we move the time forward step by step, updating the displacement and velocity at each step, until we reach $T$.

As mentioned above, we are going to assume the velocity to be constant within each subinterval. The constant velocity within a subinterval $T_i$ is given by the velocity that it has at the beginning of the subinterval, i.e., $\vec{\tilde{v_{i-1}}}$. Thus, the displacement at the end of the subinterval is given by (try to think why!):

$\vec{\tilde{x_{i}}} = \vec{\tilde{x_{i-1}}} +  \vec{\tilde{v_{i-1}}} \cdot \Delta t$

On the other hand, at the end of the subinterval, the velocity is updated by $-9.8 \cdot \Delta t \frac{m}{s}$, i.e., the projectile is accelerated at a uniform rate of $-9.8 \frac{m}{s^s}$. Thus, we have that (try to think why!):

$\vec{\tilde{v_{i}}} = \vec{\tilde{v_{i-1}}} - 9.8 \cdot \Delta t$

### The simulation code

Let see how we can turn the ideas so far into Python code. 

We are going to use three NumPy 1D arrays in the code:

* An array called `t` that will track the time evolution. In particular, `t[i]` will store $t_i$.
* An array called `x_approx` that will track the evolution of the approximate displacements. In particular, `x_approx[i]` will store $\vec{\tilde{x_{i}}}$.
* An array called `v_approx` that will track the evolution of the approximate velocities. In particular, `v_approx[i]` will store $\vec{\tilde{v_{i}}}$.

><span style="color:red">**Task 5.**</span> The code cell below provides an initial starting point with the overall structure of the simulation code. 
 1. Complete it!
 2. By looking at the plot that the code generates, does the projectile reach the ground after $T=5$ seconds?
 3. If not, when it does it approximately hit the ground? (tweak the value of `T`)

In [ ]:
# Your answer to Task 5.1 goes here

#Import external packages into current program
import numpy
import matplotlib.pyplot as plot
%matplotlib inline

# Simulation of a projectile thrown up from the ground level
N  = 10    # Number of time subintervals
T  = 5     # Total simulation time (s)
g  = -9.8  # Uniform gravitational acceleration (m/s^2)
dt = __    # Time-step (s)

t        = __ # Generate a 1D array of the appropriate size populated with zeros
x_approx = __ # Generate a 1D array of the appropriate size populated with zeros
v_approx = __ # Generate a 1D array of the appropriate size populated with zeros
    
t[0]        = 0.0  # Initial time (s)
x_approx[0] = 0.0  # Initial displacement (m)
v_approx[0] = 25.0 # Initial velocity (m/s)

#Time evolution loop
for __:
    __  #Update displacement
    __  #Update velocity
    __  #Update time
    
    
plot.plot(t, x_approx, '-x')
plot.xlabel('Time elapsed after projectile throwing (s)')
plot.ylabel('Approximate Displacement (m)')
plot.grid()
plot.show()

<span style="color:red">**Write your answers to Task 5.2 and 5.3 in this cell.**</span>

><span style="color:red">**Task 6.**</span> Copy and paste the last five lines of code of the previous cell (i.e., those starting with `plot.` in the cell right below. Modify them to plot `v_approx` as a function of `t`. Change the `xlabel` accordingly. When and at which height (approximately) the projectile starts falling down?

In [ ]:
# Code to solve Task 6 goes here

<span style="color:red">**Write your answers to questions in Task 6 in this cell.**</span>

### How accurate is our approximation?

A natural question that arises is: how accurate is the approximation of the actual physics that we are computing? Let us evaluate that!

><span style="color:red">**Task 7.**</span> Write two functions `compute_exact_displacements` and `compute_exact_velocities` that given an array which stores several points in time, returns an output array of the same size of the input array with the exact displacements and velocities at such points in time. Several hints and remarks:
* Use the analytical expressions provided above for $\vec{x}(t)$ and $\vec{v}(t)$.
* Think about the interface of the functions. Apart from the input array with the times, which parameters do the functions need?
* Recall from above that you can perform calculations on every element in the vector with a single statement (i.e., you can exploit vectorization with Numpy). For example, you can add or multiply an scalar by an array, or take of the power of an array.
* Use the two code cells provided below to test your functions. Note that in these cells we are generating a large number of different times (i.e., a small space among times passed to the `arange` function) in order to have an accurate plot of the exact displacement and velocities. Also note that we are assuming that `v_approx[0]` stores the initial velocity.

In [ ]:
# Code to solve Task 7 goes here

In [ ]:
# Code to test Task 7 (displacements)

# Space among times
dt_exact=T/1000.0

# Generate times in which we will evaluate the exact displacements and velocities
t_exact=numpy.arange(0.0,T+dt_exact,dt_exact)

x_exact=compute_exact_displacements(v_approx[0],t_exact)
v_exact=compute_exact_velocities(v_approx[0],t_exact)

plot.plot(t_exact, x_exact)
plot.xlabel('Time elapsed after projectile throwing (s)')
plot.ylabel('Exact Displacement (m)')
plot.grid()
plot.show()

In [ ]:
# Code to test Task 7 (velocities)
plot.plot(t_exact, v_exact)
plot.xlabel('Time elapsed after projectile throwing (s)')
plot.ylabel('Exact velocity (m)')
plot.grid()
plot.show()

><span style="color:red">**Task 8.**</span> Use the code provided in the cells below in order to evaluate visually the difference among the exact and approximate displacements and velocities. Start with $N=2$, and try several increasing values of $N$, e.g., $N=5$, $N=10$, $N=20$, and $N=100$. Note that you first have to execute the cell of Task 5, and then execute the two cells below each time that you increase the value of $N$. Can you describe in words what happens with the displacements as you increase $N$? And with the velocities?

In [ ]:
# Code to answer Task 8
plot.plot(t, x_approx, 'x', label='Approximate')
plot.plot(t_exact, x_exact, label='Exact')
plot.xlabel('Time elapsed after projectile throwing (s)')
plot.ylabel('Exact displacement versus approximate displacement (m)')
plot.grid()
plot.legend()
plot.show()

In [ ]:
# Code to answer Task 8
plot.plot(t, v_approx, 'x', label='Approximate')
plot.plot(t_exact, v_exact, label='Exact')
plot.xlabel('Time elapsed after projectile throwing (s)')
plot.ylabel('Exact velocity versus approximate velocity (m/s)')
plot.grid()
plot.legend()
plot.show()

<span style="color:red">**Write your answers to questions in Task 8 in this cell.**</span>

# Learning outcomes

In this workshop you built a simple simulator that approximates the displacement of a projectile thrown not too fast up from the ground level in a uniform gravitational field. 

Besides, you have also visually evaluated how accurate such approximation is with respect to the actual displacement provided by the basic laws of Newtonian mechanics. 

The computational pattern of the code written in this tutorial is shared by many actual numerical simulations. A time interval is "discretized", i.e., split into smaller subintervals, and the evolution of a physical phenomenon is approximated as the time moves forward.

In the next tutorial, we will move to simulating more interesting problems, and consider 2D motion.

The coding skills you have learnt include:
* Generating, populating, and manipulating 1D arrays.
* How `for` loops can be used to iterate sub-indices in mathematical notation.
* Plotting 1D arrays data.